In [97]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from time import sleep
from tqdm import tqdm


creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)

print('hi')
print(sp)


hi


In [101]:
def get_album_genre(album_name, artist_name):
  
    q = "artist:{} album:{}".format(artist_name, album_name)
    
    results = sp.search(q, limit=1, offset=0, type='album')

    if len(results['albums']['items']) > 0:
        
        # try grabbing genre through album
        album_id = results['albums']['items'][0]['id']
        album = sp.album(album_id)
        genres = album['genres']
        print("album genre: " + str(genres))
        # first one category, second genre --- can change this if want
        if len(genres) >= 2:
            return genres[0], genres[1]
        elif len(genres) >= 1:
            return genres[0], genres[0]
        else:
            # try grabbing genre through artist
            artist_id = results['albums']['items'][0]['artists'][0]['id']
            artist_info = sp.artist(artist_id)
            genres = artist_info['genres']
            print("artist genre: " + str(genres))
            # first one category, second genre --- can change this if want
            if len(genres) >= 2:
                return genres[0], genres[1]
            elif len(genres) >= 1:
                return genres[0], genres[0]
            else:
                return None, None
    else:
        return None, None

In [102]:
artist_name = "Linkin Park"
song_name = "Meteora"
try:
    res = get_album_genre(song_name, artist_name)
    print(res)
except Exception as e:
    print(str(e))

album genre: []
artist genre: ['alternative metal', 'nu metal', 'post-grunge', 'rap metal', 'rock']
('alternative metal', 'nu metal')


In [105]:

cddb_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_3.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/4_year_repair/cddb_tracks.tsv', sep='\t')


cddb_df_trim = cddb_df[cddb_df['genre'] == 'Data']
cddb_df_trim = cddb_df_trim[['artist', 'title', 'id']]
print(cddb_df_trim)

                            artist   
2                        No Return  \
4                          Emanuel   
14                           Keedy   
25                 Jazz Signatures   
26             Bernadã¨te Bidã ude   
33            Sportfreunde Stiller   
34         Rennie Pilgrim Presents   
38                            Deux   
40                         Various   
41                 Matsuka Phonics   
43                        Robotika   
48                  Rolling Stones   
49    Ã¤ã®ã«ã¥ã°ã¤ Ã¹ã¬ Ã¤ã¢ã¸ã¥ã¡   
61           Â¿â©ã‡ã _â½âºã„ã‰ã„â¡   
62                    Diversetion)   
9332                    Pink Floyd   

                                                  title      id  
2                                       Self Mutilation  100002  
4                                            Felicidade  100004  
14                                     Chase The Clouds  100015  
25                                      Jazz Signatures  100028  
26                     

In [106]:
# lists are much faster than dataframes, converting
cddb_list = cddb_df_trim.values.tolist()
tracks_list = tracks_df.values.tolist()

print(f'num null years: ${len(cddb_list)}')

num null years: $16


In [107]:
def start_repair(cd_list):
    
    total_rows = len(cd_list)
    for cddb_row in tqdm(cd_list, total=total_rows, desc='Processing rows', ncols=100):
    #for cddb_row in cd_list:
  
        artist_name = cddb_row[0]
        album_name = cddb_row[1]

        category, genre = get_album_genre(album_name, artist_name)
        if category:
            # update cddb_df directly
            idx_list = cddb_df.index[cddb_df['id'] == cddb_row[2]].tolist()
            idx = idx_list[0]
            cddb_df.at[idx,'category'] = fix
            cddb_df.at[idx,'genre'] = fix
            print("fixed - :" + str(cddb_row) + "fix: " + str(fix))
    

In [108]:
start_repair(cddb_list)

Processing rows:  12%|█████▉                                         | 2/16 [00:00<00:03,  4.56it/s]

album genre: []
artist genre: ['classic portuguese pop']
fixed - :['Emanuel', 'Felicidade', 100004]idx: 4


Processing rows:  44%|████████████████████▌                          | 7/16 [00:01<00:01,  6.06it/s]

album genre: []
artist genre: ['german indie', 'german indie rock', 'german pop', 'german rock', 'munich indie']
fixed - :['Sportfreunde Stiller', 'Die Gute Seite', 100037]idx: 33


Processing rows:  56%|██████████████████████████▍                    | 9/16 [00:01<00:01,  5.65it/s]

album genre: []
artist genre: ['dfw rap']
fixed - :['Deux', 'Forever', 100042]idx: 38


Processing rows:  81%|█████████████████████████████████████▍        | 13/16 [00:02<00:00,  5.85it/s]

album genre: []
artist genre: []


Processing rows: 100%|██████████████████████████████████████████████| 16/16 [00:02<00:00,  6.24it/s]


In [109]:
#save to output
cddb_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_4.tsv', sep="\t", index=False)
tracks_df.to_csv('../../cleaning_stages/5_genre_repair/cddb_tracks.tsv', sep="\t", index=False)